In [23]:
import numpy as np
import math

import sys
import types

import RPI
import NBS
import RSD
import RPIk

np.set_printoptions(precision=3)

In [24]:
# Returns size x size matrices that, in theory, should yield an RSD:NBS worst case approximation of size 
# Takes the size of matrix (# of agents and items)
def n_approx_val_generator(size):
    
    eps = sys.float_info.epsilon
    V = np.zeros(shape=(size, size))
    
    V[0][0] = 1
    for i in range(1, size):
        V[i][0] = 1
        V[i][i] = float(1-eps)
    
    return V

In [25]:
def i_mat(size):
    V = np.zeros((size, size))
    for i in range(size):
        V[i][i] = 1
    return V

In [26]:
def monte_carlo_RPI(V, n):
    max_size = max(V.shape[0], V.shape[1])
    p = np.zeros((max_size, max_size))
    for i in range(n):
        p = np.add(p, RPI.RPI(V, 4))
        
    ax_0 = []
    for i in range(V.shape[0], p.shape[0]):
        ax_0.append(i)
    p = np.delete(p, ax_0, 0)
    
    ax_1 = []
    for j in range(V.shape[1], p.shape[1]):
        ax_1.append(j)
    p = np.delete(p, ax_1, 1)
        
    return p/n

In [27]:
def monte_carlo_RPIk(V, n):
    max_size = max(V.shape[0], V.shape[1])
    p = np.zeros((max_size, max_size))
    for i in range(n):
        p = np.add(p, RPIk.RPIk(V, 1))
        
    ax_0 = []
    for i in range(V.shape[0], p.shape[0]):
        ax_0.append(i)
    p = np.delete(p, ax_0, 0)
    
    ax_1 = []
    for j in range(V.shape[1], p.shape[1]):
        ax_1.append(j)
    p = np.delete(p, ax_1, 1)
        
    return p/n

In [28]:
# Returns expected value of agent/item utility matrix determined empirically
# Takes value matrix, optionally number of iterations (default 1000)
def monte_carlo_RSD(V, n):     
    p = np.zeros(V.shape)
    for i in range(n):
        p = np.add(p, RSD.RSD(V))
    return p/n

In [29]:
# Takes num agents, num items, distribution for agent values to be drawn from, and optionally number of trials
# Returns worst-case ratio 
def worst_case_ratio(V, Func, n=1000):
    if not(type(V) == np.ndarray):
        raise Exception("V must be a numpy.ndarray, currently type: ", type(V))
    if not isinstance(Func, types.FunctionType):
        raise Exception("Func must be a function, currently type: ", type(Func))
    if np.ndim(V) != 2:
        raise Exception("V must be a 2-D np.ndarray. Current shape: ", vals.shape)
        
    O = np.zeros(V.shape[0])
    
    P_NBS = NBS.NBS(V,O, np.ones(V.shape[1]))
    if P_NBS is None: 
        raise Exception("NBS returned None")
        
    P_Func = Func(V, n)
        
    U_NBS = np.sum(np.multiply(V, P_NBS), axis=1)
    U_Func = np.sum(np.multiply(V, P_Func), axis=1)
    
    U_ratio = np.divide(U_NBS, U_Func)
    return np.amax(U_ratio)

In [30]:
def same_pref(i, j):
    v = np.zeros(shape=(i,j))
    v[0] = np.random.rand(1, j)
    for k in range(1, i):
        v[k] = v[k-1]
    return v

In [8]:
for i in range(5, 25):
    v = i_mat(i)
    wcr_RPIk = worst_case_ratio(v, monte_carlo_RPIk)
    print("WCR RPIk: ", wcr_RPIk)
    print("\n __________________________ \n")
  

WCR RPIk:  1.000000000152404

 __________________________ 

WCR RPIk:  1.0000000002538534

 __________________________ 

WCR RPIk:  1.0000000003197442

 __________________________ 

WCR RPIk:  1.000000000301612

 __________________________ 

WCR RPIk:  1.0000000002235079

 __________________________ 

WCR RPIk:  1.000000000137767

 __________________________ 

WCR RPIk:  1.0000000003136975

 __________________________ 

WCR RPIk:  1.000000000148851

 __________________________ 

WCR RPIk:  1.0000000002022125

 __________________________ 

WCR RPIk:  1.0000000001297642

 __________________________ 

WCR RPIk:  1.000000000142609

 __________________________ 



KeyboardInterrupt: 

In [ ]:
print("V: \n", v)
p = RSD.RSD(v)
print("RSD: \n", p)

In [ ]:
max_size = 11 #exclusive
min_size = 5 #inclusive
assert(min_size >= 4)
assert(max_size > min_size)

wcr_RPIk = np.zeros(shape=(max_size-min_size, max_size-min_size))
wcr_RPI = np.zeros(shape=(max_size-min_size, max_size-min_size))
wcr_RSD = np.zeros(shape=(max_size-min_size, max_size-min_size))

# i is num agents, j is num courses
for i in range(min_size, max_size):
    for j in range(min_size, max_size):
        v = RPI.pref_att(i, j, 0.5)
        wcr_RPIk[i-min_size][j-min_size] = worst_case_ratio(v, monte_carlo_RPIk)
        wcr_RPI[i-min_size][j-min_size] = worst_case_ratio(v, monte_carlo_RPI)
        wcr_RSD[i-min_size][j-min_size] = worst_case_ratio(v, monte_carlo_RSD)
        
        

In [ ]:
print(wcr_RPI)

In [ ]:
print(wcr_RSD)

In [ ]:
print(wcr_RPIk)

In [35]:
min_size = 5
max_size = 25
wcr_RPIk = np.zeros(max_size-min_size)
wcr_RPI = np.zeros(max_size-min_size)
wcr_RSD = np.zeros(max_size-min_size)

# i is num agents, j is num courses
for i in range(min_size, max_size):
        v = same_pref(i, i)
        
        rpi = worst_case_ratio(v, monte_carlo_RPI)
        print("WCR RPI: \n", rpi)
        wcr_RPI[i-min_size] = rpi
        
        rpik = worst_case_ratio(v, monte_carlo_RPIk)
        print("WCR RPIk: \n", rpik)
        wcr_RPIk[i-min_size] = rpik
        
        rsd = worst_case_ratio(v, monte_carlo_RSD)
        print("RSD: \n", rsd)
        wcr_RSD[i-min_size] = rsd
        
        print("\n---------------------------\n")
        

WCR RPI: 
 1.0165995531483767
WCR RPIk: 
 1.0053605154431542
RSD: 
 1.0183191781217966

---------------------------

WCR RPI: 
 1.0118672631230008
WCR RPIk: 
 1.0018506980266348
RSD: 
 1.0124033480576609

---------------------------

WCR RPI: 
 1.0130745438976971
WCR RPIk: 
 1.0019421409229337
RSD: 
 1.017170038347828

---------------------------

WCR RPI: 
 1.0178859879258622
WCR RPIk: 
 1.0025266653756586
RSD: 
 1.0193043018128456

---------------------------

WCR RPI: 
 1.0204013591497805
WCR RPIk: 
 1.003885491271047
RSD: 
 1.031073634604771

---------------------------

WCR RPI: 
 1.0231534002107014
WCR RPIk: 
 1.0018409627918168
RSD: 
 1.0283598620309282

---------------------------

WCR RPI: 
 1.014194254154847
WCR RPIk: 
 1.0030549169084124
RSD: 
 1.0336818185027918

---------------------------

WCR RPI: 
 1.0156946741100847
WCR RPIk: 
 1.0025667358703916
RSD: 
 1.032629453701738

---------------------------



KeyboardInterrupt: 

In [ ]:
print(wcr_RPIk)
print(wcr_RSD)